In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [3]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory+automation' # topic of the search
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [4]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are Senior Press Editor for technology press. When you writting a news report, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Avoid homogeneous ideas
        - Be creative and explorative in creating eye-catching headline
        """
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```

    You want to write news report about what problems does future factory automation would face. Kindly follow these guidelines
    - Generate a interesting headline
    - Be realistic
    - Do consider about challenges in business when making decision
    - Keep it in 50 words
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

"Robotic Revolution or Risk? Aniai's Leap into Automated Cooking Faces Production Puzzles"

As Aniai gears up with $12M funding to launch Factory One, the robotic cooking industry faces challenges of scalability, tech integration, and the balancing act between innovation and reliable supply chain management amidst global expansion ambitions.

---

Problem:

"Check-In to the Future: Automation's Next Hurdle in Hospitality" 

Facing integration complexities and cybersecurity threats, the leap to fully automated guest services in hotels poses significant challenges. Businesses must balance technological advancements with personalized guest experiences, all while ensuring data protection and seamless operation across diverse systems.

---

Problem:

"Revolutionizing Medicine: The Hurdles Ahead for Cellares' $255M Automation Dream" 

As Cellares propels forward with its $255M funding for Smart Factories, the journey teems with challenges: scaling up production while ensuring cGMP 

In [5]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria. Please be very critical in your ratings and only give a 3 rating when it is exceptionally good:
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 20 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

"Robotic Revolution or Risk? Aniai's Leap into Automated Cooking Faces Production Puzzles"

- Size of user group: 2 (The automated cooking industry targets a medium-sized, growing user base.)
- Relevancy of pain point: 3 (Scalability and tech integration are highly relevant to the factory automation sector.)
- How specific is the problem: 2 (Challenges are somewhat specific to the robotic cooking industry, yet somewhat common in tech scale-ups.)

Overall score: 7/9

Justification: The issue strikes a chord in the automation sphere, affecting a substantial, though not universal, audience.

---

Problem rating:

"Check-In to the Future: Automation's Next Hurdle in Hospitality"

- Size of user group: 2 (Medium target group, as it specifically targets the hospitality industry.)
- Relevancy of pain point: 3 (Very relevant to users, directly impacting hotel operations and guest satisfaction.)
- How specific is the problem: 3 (Specific and not generic problem, addressing inte

In [ ]:
# third agent to give best problem 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
       - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
 
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")


